In [1]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import glob
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as trns
import torch.nn as nn
from pytorch_lightning.callbacks import ModelCheckpoint, Callback

from torchmetrics import JaccardIndex
from sklearn.model_selection import train_test_split

import os
import albumentations as A

from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
from torch.optim import lr_scheduler
import segmentation_models_pytorch as smp
import pytorch_lightning as pl
import random




In [2]:
print(torch.__version__)
print(torch.version.cuda)  # Should print the version of CUDA PyTorch is using
print("cuDNN version:", torch.backends.cudnn.version())
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the device to GPU if available, otherwise use CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

2.7.1+cu118
11.8
cuDNN version: 90100
cuda


In [3]:
# The spawn multiprocessing method used on Windows struggles to transfer code defined dynamically within a notebook's __main__ scope to new worker 
# processes. Moving the Dataset definition to an importable .py file provides a reliable way for the worker processes to access the necessary code 
# definition via standard Python imports.
from utils import *   


# Data Loading
print("Scanning for files...")
image_files = []
mask_files = []
# path to the directory containing labeled data
# ├── Folder1\
# │   ├── images\
# │   │   ├── tile_0_image.tiff
# │   │   ├── ...
# │   ├── masks\
# │   │   ├── tile_0_mask.tiff
# │   │   ├── ...
# ├── Folder2\
# │   ├── images\
# │   │   ├── tile_0_image.tiff
# │   │   ├── ...
# │   ├── masks\
# │   │   ├── tile_0_mask.tiff
# │   │   ├── ...
# └── ... (other main folders)
data_root = r"C:\Annotated Dataset\L2A Data\Tiles10and20mbandsSubsBathy_G0Percent_stride256\*"  #C:\Dataset\Ultimate10and20mbandsSubsBathy\*    C:\Dataset\Ultimate10and20mbandsSubsBathyReRevised\*
folders = glob.glob(data_root)
folders = sorted(folders)
if not folders:
    print(f"Warning: No folders found at {data_root}")



# Selecting 70% of the folders for training, 15% for validation, and 15% for test (did this to use geographically independent data for tr, val, and test.
# Create a list of all folder paths
all_folders = [f for f in folders if os.path.isdir(f)]




random.seed(42)
random.shuffle(all_folders)

num_folders = len(all_folders)
train_split = int(0.70 * num_folders)
val_split = int(0.15 * num_folders)


# ############ FIX THIS ###############
# train_split = 4
# val_split = 1



train_folders = all_folders[:train_split]
val_folders = all_folders[train_split:train_split + val_split]
test_folders = all_folders[train_split + val_split:]

def get_files_from_folders(folder_list):
    image_paths = []
    mask_paths = []
    for f in folder_list:
        image_subfolder = os.path.join(f, "images")
        mask_subfolder = os.path.join(f, "masks")
        if os.path.isdir(image_subfolder) and os.path.isdir(mask_subfolder):
            image_paths.extend(glob.glob(os.path.join(image_subfolder, "*")))
            mask_paths.extend(glob.glob(os.path.join(mask_subfolder, "*")))
        else:
            print(f"Warning: Folder {f} did not contain 'images' and 'masks' subfolders.")
    return image_paths, mask_paths

X_train_paths, y_train_paths = get_files_from_folders(train_folders)
X_val_paths, y_val_paths = get_files_from_folders(val_folders)
X_test_paths, y_test_paths = get_files_from_folders(test_folders)

print(f"Found {len(X_train_paths)} train images and {len(y_train_paths)} train masks in {len(train_folders)} folders.")
print(f"Found {len(X_val_paths)} validation images and {len(y_val_paths)} validation masks in {len(val_folders)} folders.")
print(f"Found {len(X_test_paths)} test images and {len(y_test_paths)} test masks in {len(test_folders)} folders.")

# Basic check if counts are reasonable
if len(X_train_paths) != len(y_train_paths) or \
   len(X_val_paths) != len(y_val_paths) or \
   len(X_test_paths) != len(y_test_paths):
    raise ValueError("Mismatch in the number of image and mask files in one or more splits.")


print("\nTrain folders:")
for folder in train_folders:
    print(folder)

print("\nValidation folders:")
for folder in val_folders:
    print(folder)

print("\nTest folders:")
for folder in test_folders:
    print(folder)




# # For random splitting, uncomment the following
for f in folders:
    files = glob.glob(f"{f}/*")
    # Basic check if expected subfolders/files exist
    if len(files) >= 2:
        image_files.extend(glob.glob(f"{files[0]}/*"))
        mask_files.extend(glob.glob(f"{files[1]}/*"))
    else:
        print(f"Warning: Folder {f} did not contain expected structure.")

print(f"Found {len(image_files)} images and {len(mask_files)} masks.")

# Ensure we have data before splitting
if not image_files or not mask_files or len(image_files) != len(mask_files):
    raise ValueError("Mismatch in image/mask files or no files found. Check paths.")

# Proceed with splitting only if files were found
X_train_paths, X_temp_paths, y_train_paths, y_temp_paths = train_test_split(
    image_files, mask_files, test_size=0.003, random_state=42 # Use random_state!             0.3
)
X_val_paths, X_test_paths, y_val_paths, y_test_paths = train_test_split(
    X_temp_paths, y_temp_paths, test_size=0.005, random_state=42 # Use same random_state               0.5
)

print(f"Train samples: {len(X_train_paths)}, Validation samples: {len(X_val_paths)}, Test samples: {len(X_test_paths)}")




# Use data from a few folders for testing and shuffle the rest for tr and val (80-20)
# --- Step 1: Assign second folder as test ---
if len(folders) < 2:
    raise ValueError("At least 2 folders are required to assign one for testing.")

test_indices = [6]  # Use whatever indices you want here eg [9,1,3]
test_folders = [folders[i] for i in test_indices]

# Get image/mask paths for test folder
X_test_paths, y_test_paths = get_files_from_folders(test_folders)

# Exclude the actual test folder from train/val
# train_val_folders = [f for i, f in enumerate(folders) if i != test_index]
train_val_folders = [f for i, f in enumerate(folders) if i not in test_indices]

# Gather all image/mask file paths
X_all_paths, y_all_paths = get_files_from_folders(train_val_folders)

# --- Step 3: Shuffle the image/mask files together (zip to keep them aligned) ---
combined = list(zip(X_all_paths, y_all_paths))
random.seed(42)
random.shuffle(combined)
X_all_paths[:], y_all_paths[:] = zip(*combined)

# --- Step 4: Split 80% for training, 20% for validation ---
split_idx = int(0.8 * len(X_all_paths))
X_train_paths = X_all_paths[:split_idx]
y_train_paths = y_all_paths[:split_idx]
X_val_paths = X_all_paths[split_idx:]
y_val_paths = y_all_paths[split_idx:]

# --- Reporting ---
print(f"Train samples: {len(X_train_paths)}")
print(f"Validation samples: {len(X_val_paths)}")
print(f"Test samples: {len(X_test_paths)}")

# --- Sanity check ---
if len(X_train_paths) != len(y_train_paths) or \
   len(X_val_paths) != len(y_val_paths) or \
   len(X_test_paths) != len(y_test_paths):
    raise ValueError("Mismatch in image and mask counts in one or more splits.")

print("\nTest folder(s):")
print(test_folders)



Scanning for files...
Found 515 train images and 515 train masks in 7 folders.
Found 62 validation images and 62 validation masks in 1 folders.
Found 385 test images and 385 test masks in 2 folders.

Train folders:
C:\Annotated Dataset\L2A Data\Tiles10and20mbandsSubsBathy_G0Percent_stride256\20230819T192911_20230819T193100_T09UWT
C:\Annotated Dataset\L2A Data\Tiles10and20mbandsSubsBathy_G0Percent_stride256\20220806T191919_20220806T192707_T09UYQ
C:\Annotated Dataset\L2A Data\Tiles10and20mbandsSubsBathy_G0Percent_stride256\20210727T191911_20210727T192721_T09UXQ
C:\Annotated Dataset\L2A Data\Tiles10and20mbandsSubsBathy_G0Percent_stride256\20230902T190921_20230902T191805_T10UDU
C:\Annotated Dataset\L2A Data\Tiles10and20mbandsSubsBathy_G0Percent_stride256\20230804T192909_20230804T192942_T09UXS
C:\Annotated Dataset\L2A Data\Tiles10and20mbandsSubsBathy_G0Percent_stride256\20230816T191911_20230816T192348_T10UCA
C:\Annotated Dataset\L2A Data\Tiles10and20mbandsSubsBathy_G0Percent_stride256\20230

In [4]:
# Augmentation:  The function returns an instance of the A.Compose class. This instance is an object that, when called with an image and mask 
# (as keyword arguments image= and mask=), will apply the defined sequence of augmentations to them and return a dictionary containing the 
# augmented image and mask.
# The get_training_augmentation() function is designed to create the augmentation pipeline. It's not meant to directly perform the augmentation on a 
# specific image and mask. The A.Compose object that it returns is the callable that takes the image and mask as input later, when it's called within 
# the __getitem__ method inside the SatelliteDataset class
def get_training_augmentation():
    return A.Compose(
        [
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            # A.RandomRotate90(p=0.5)
        ]
    )


In [5]:
CLASSES = ["kelp"]

In [6]:
# Calculate the mean and std for normalization using training data
# Create a DataLoader for the training data (without augmentation)
temp_train_dataset = SatelliteDataset(
    image_paths=X_train_paths,
    mask_paths=y_train_paths,
    classes=CLASSES,
    augmentation=None,
    mean=None,
    std=None
)
# We use DataLoader to load images in batches, which is much more efficient for I/O and can leverage multiple CPU cores (num_workers).
temp_train_loader = DataLoader(temp_train_dataset, batch_size=32, shuffle=False, num_workers=8, pin_memory=True) # Adjust batch_size and num_workers as needed

# Initialize accumulators for mean and std
sum_per_channel = None
sum_sq_per_channel = None
total_pixels = 0
num_channels = None

# Iterate through the DataLoader to calculate mean and std
for batch_idx, (images, _) in enumerate(temp_train_loader):
    # images shape: (batch_size, C, H, W)
    batch_size, C, H, W = images.shape
    images = images.numpy() # Move to NumPy for easier calculations

    if num_channels is None:
        num_channels = C

    # Sum across batch, height, and width for each channel
    current_sum = np.sum(images, axis=(0, 2, 3)) # Shape: (C,)
    current_sum_sq = np.sum(images ** 2, axis=(0, 2, 3)) # Shape: (C,)
    current_pixels = batch_size * H * W

    if sum_per_channel is None:
        sum_per_channel = current_sum
        sum_sq_per_channel = current_sum_sq
    else:
        sum_per_channel += current_sum
        sum_sq_per_channel += current_sum_sq
    total_pixels += current_pixels

# Calculate the mean and std
mean_per_channel = sum_per_channel / total_pixels
std_per_channel = np.sqrt((sum_sq_per_channel / total_pixels) - (mean_per_channel ** 2))

# print("Calculated Mean per channel:", mean_per_channel)
# print("Calculated Std per channel:", std_per_channel)


In [7]:
print("Calculated Mean per channel:", mean_per_channel)
print("Calculated Std per channel:", std_per_channel)

Calculated Mean per channel: [ 1.9271057e+02  2.4526277e+02  1.3288297e+02  9.1582831e+02
  2.9818857e+02  7.2954968e+02  8.6662543e+02  9.5027246e+02
  4.1283145e+02  2.0912796e+02  1.3694173e+00 -4.0696268e+00
  1.7028417e-02  2.0986709e-01 -2.0986709e-01  1.4291023e+07
  8.4679022e-02]
Calculated Std per channel: [1.5137001e+02 2.0824329e+02 1.9422487e+02 1.2421727e+03 3.6834180e+02
 9.6155676e+02 1.1546597e+03 1.2775868e+03 5.8319891e+02 3.3291693e+02
 1.3369064e+00 2.1212910e+02 6.7632693e-01 7.2675657e-01 7.2675657e-01
 1.8433802e+10 4.1082326e-01]
